In [ ]:
[1]*10**10

# Install latest YOLOv5 version and dependencies. Store the working directory

In [ ]:
%cd Base/Directory
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -qr yolov5/requirements.txt  # install dependencies (ignore errors)
%cd yolov5

import torch
from IPython.display import Image, clear_output  # to display images


clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.10.0+cu111 CPU


# Loading Training, testing and Validation datasets. Keep the training data in yolo format, i.e., (centre_x,centre_y,width,height) in normalized format

In [ ]:
# !apt install unzip
from zipfile import ZipFile
zf = ZipFile('Path/to/train.zip', 'r')
zf.extractall('Base/Folder')
zf = ZipFile('Path/to/valid.zip', 'r')
zf.extractall('Base/Folder')
zf = ZipFile('Base/test_video.zip', 'r')
zf.extractall('Base/Folder')
zf.close()

In [ ]:
# Train YOLOv5s on coco128 for 3 epochs /content/drive/MyDrive/YOLO/yolov5/datasets/MOT/valid.txt
!python train.py --img 256 --batch 16 --epochs 300 --data data.yaml --weights yolov5s.pt

/content/drive/MyDrive/YOLO/yolov5
train: weights=yolov5s.pt, cfg=, data=farah.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=300, batch_size=16, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds
YOLOv5 🚀 2021-12-3 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7,

# Making performance curves using TensorBoard

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir runs 

In [ ]:
 %cd path/to/YOLOv5/folder
!python detect.py --weights runs/train/exp0/weights/best.pt --img 640 --conf 0.1 --source base/directory/Test_video.mp4                      

Cloning Deepsort

In [ ]:
!git clone --recurse-submodules https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch.git

/content/drive/MyDrive/YOLO
fatal: destination path 'Yolov5_DeepSort_Pytorch' already exists and is not an empty directory.


In [ ]:
%cd Base/Folder/Yolov5_DeepSort_Pytorch
!pip install -r requirements.txt

/content/drive/MyDrive/YOLO/Yolov5_DeepSort_Pytorch
     |████████████████████████████████| 596 kB 5.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!python track.py --source path/to/test/frames --yolo_weights path/to/best.pt --classes 0 --classes 0 --save-txt   # video --save-txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 '''Here to save results in text form, I have made changes in the visualize_boxes_and_labels_on_image_array function
  by putting class and box value in the print statement and saving them directly form output screen.'''
import numpy as np
from PIL import Image
import os
import glob
import cv2
import matplotlib.pyplot as plt
import warnings
from object_detection.utils import vis4 #changed version of visualization_utils, where print statement is added at 1241line number
from object_detection.utils import visualization_utils
import tensorflow.python.ops.numpy_ops.np_config

warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings
IMAGE_PATHS = 'path/to/test/frames/*.jpg'

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in glob.glob(IMAGE_PATHS):
    filename = os.path.basename(image_path)

    print('Running inference for {}...\n'.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)
    
    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections
    # print(detections['num_detections'])
    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()
    boxes = detections['detection_boxes']
    lboxes = len(detections['detection_boxes'])
    lclasses = len(detections['detection_classes'])
    #print(bgoxes,lboxes,lclasses)
    vis4.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)
    
    plt.figure()
    plt.imshow(image_np_with_detections)
    plt.show()
    data = Image.fromarray(image_np_with_detections)
    data.save('<Path to out images files>'+filename)
    print('Done')
    
    
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

   
